In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import tensorflow as tf
from datetime import datetime, timezone

# spectral analysis
from scipy import signal
from scipy.signal import periodogram as periodogram_f
from scipy.fft import fftfreq, fftshift
from scipy.fft import fft, ifft, fft2, ifft2

In [ ]:
# with pandas 2.0, one could use date_format='%Y-%m-%d %H:%M:%S%z', but that's not yet available on Arch Linux
solar_ts=pd.read_csv("data/energy_charts.csv", sep=",", header=0)#date_format='%Y-%m-%d %H:%M:%S%z')#parse_dates={"date": ["Datum"]})

In [ ]:
print(solar_ts['Datum'])

In [ ]:
solar_ts['Datum']=pd.to_datetime(solar_ts['Datum'], format='%Y-%m-%dT%H:%M%z', utc=True)
solar_ts=solar_ts.set_index(keys="Datum",drop=True)
solar_ts.plot()

In [ ]:
adfresult = adfuller(solar_ts[2:30000])
print(adfresult[0])
print(adfresult[1])

In [ ]:
# see https://stackoverflow.com/questions/30379789/plot-pandas-data-frame-with-year-over-year-data
pv = pd.pivot_table(solar_ts, index=solar_ts.index.dayofyear, columns=solar_ts.index.year,
                    values='Leistung', aggfunc='sum')
pv.plot(cmap="Grays")

In [ ]:
# see https://stackoverflow.com/questions/30379789/plot-pandas-data-frame-with-year-over-year-data
pv = pd.pivot_table(solar_ts, index=solar_ts.index.month, columns=solar_ts.index.year,
                    values='Leistung', aggfunc='sum')
pv.plot(cmap="Grays")

In [ ]:
# An example of a gap in the data
# TODO: Also, there is duplicate data here that pandas duplicated-function will not find...?
solar_ts.index[5660:5680]

In [ ]:
pd.Series(solar_ts.index.duplicated()).value_counts()

In [ ]:
(pd.Series(solar_ts.index[5660:5680]).diff())

In [ ]:
# Those values need imputation!
pd.date_range(solar_ts.index.min(), solar_ts.index.max(), freq='15Min').difference(solar_ts.index)

In [ ]:
# This add NaN as value for the missing indices, we can impute this later.
solar_ts = solar_ts.resample("15Min").first()
# As only a few values need imputation, so the choice of the imputation algorithm does not matter much.
solar_ts = solar_ts.interpolate(method="time")
# Only now can we infer a frequency.
solar_ts=solar_ts.asfreq(pd.infer_freq(solar_ts.index))

In [ ]:
# There are no duplicated dates, good!
# (Although, a bit questionable, see above)
np.count_nonzero(solar_ts.index.duplicated())

In [ ]:
solar_ts=solar_ts.asfreq(pd.infer_freq(solar_ts.index))

In [ ]:
solar_ts.plot()

In [ ]:
solar_ts_series = solar_ts.Leistung

In [ ]:
# Normalize
avg, dev = solar_ts_series.mean(), solar_ts_series.std()
solar_ts_series = (solar_ts_series - avg)/dev
solar_ts_series.plot()

In [ ]:
# Remove trend (TODO: compare with the approach in the Fourier series video, where they also detrend?)
solar_ts_series = solar_ts_series.diff().dropna()
solar_ts_series.plot()

In [ ]:
# Consider taking another difference: solar_ts_series = solar_ts_series.diff().dropna()
# solar_ts_series.plot()

In [ ]:
# remove increasing volatility - or (TODO: use a (G)ARCH here).
annual_volatility = solar_ts_series.groupby(solar_ts_series.index.year).std()
annual_vol_per_day = solar_ts_series.index.map(lambda d: annual_volatility.loc[d.year])
solar_ts_series_corrected_variance = solar_ts_series/annual_vol_per_day

In [ ]:
annual_volatility

In [ ]:
annual_vol_per_day

In [ ]:
solar_ts_series_corrected_variance.plot()

In [ ]:
# ritvik takes monthly means here
# why not take dayofyear?
monthly_mean = solar_ts_series_corrected_variance.groupby(solar_ts_series_corrected_variance.index.month).mean()
monthly_mean_per_day = solar_ts_series_corrected_variance.index.map(lambda d: monthly_mean.loc[d.month])

In [ ]:
solar_ts_series_corrected_variance= solar_ts_series_corrected_variance - monthly_mean_per_day

In [ ]:
solar_ts_series_corrected_variance.plot()

In [ ]:
# we only take the first few samples as my RAM explodes otherwise
adfresult = adfuller(solar_ts_series_corrected_variance[3:30000])
print(adfresult[0])
print(adfresult[1])
adfresult = adfuller(solar_ts_series_corrected_variance[120000:150000])
print(adfresult[0])
print(adfresult[1])

In [ ]:
solar_ts_series_corrected_variance=solar_ts_series_corrected_variance[~np.isnan(solar_ts_series_corrected_variance)]
solar_ts_series_corrected_variance

# some spectral analysis

In [ ]:
# ts has period 15 minutes
dt = 15*60
rate = 1/dt
periodogram = np.abs(fft(np.asarray(solar_ts_series_corrected_variance)))**2*dt/(len(solar_ts_series_corrected_variance))
frequencies = fftfreq(len(solar_ts_series_corrected_variance), d=1/rate)
frequencies
plt.plot(fftshift(frequencies), fftshift(periodogram))
plt.xlim(-0.00002, 0.0001)
plt.ylim(0, 10000)

In [ ]:
# looks (and should be!) similar to the "manual" calculation above.
# Note that in the manual calculation, we get a symmetric graph. That's to be expected (check out the videos).
frequencies, periodogram = periodogram_f(np.asarray(solar_ts_series_corrected_variance), fs=rate, window="hamming")
plt.plot(fftshift(frequencies), fftshift(periodogram))
plt.xlim(-0.00002, 0.0001)
plt.ylim(0, 10000)

In [ ]:
periodogram_as_series = pd.Series(fftshift(periodogram), index=fftshift(frequencies))
periodogram_as_series = periodogram_as_series[periodogram_as_series.index > 0]

In [ ]:
# convert index from frequencies to periods and convert the periods to hours
# TODO: is the calculation to hours correct (note that  we already specified the sampling rate during the fft!)?
periodogram_as_series.index = (1/periodogram_as_series.index)/3600

In [ ]:
plt.plot(periodogram_as_series)
plt.xlim(0,100000/3600)

In [ ]:
# TODO: use SARIMA, => detrend and remove saisonality
# Take a look the the residuals
# is the model good?
# Then, the residuals have no (p)ACF
# check QQ - WN has no heavy tails :)
# also consider: https://www.youtube.com/watch?v=4zV-ZyQHl7s

# TODO: decompose + fit SARIMA model
# before: continue with denoising :)

In [ ]:
# yet another way to calculate the FFT, from the "denoising" video.
# However, apparently, the signal is now dampened. The frequencies themselves are correct, though
# Note how damped the signal appears visually already although the y-scale is really small!
n = len(solar_ts_series_corrected_variance)
fhat = np.fft.fft(solar_ts_series_corrected_variance, n)
PSD = fhat*np.conj(fhat)/n
freq = (1/(dt*n))*np.arange(n)
L= np.arange(1, np.floor(n/2), dtype="int")
plt.plot(freq[L], PSD[L])
plt.xlim(-0.00002, 0.0001)
plt.ylim(0, 100)

In [ ]:
# Now decide on the frequencies to cut off.
plt.plot(freq, PSD)
plt.ylim(1e-2,1e5)
plt.axhline(y=1e0, color="r")
plt.semilogy()

In [ ]:
# We'll only retain frequencies with powers above the red line in the graph above.
# I chose it such that almost all high-power frequencies are retained. We get some noisy frequencies in (at the tails) but the majority is filtered.
indices = PSD > 1e0
# Filter and reconstruct the signal on the retained frequencies (reverse fourier transform)
PSDclean = PSD*indices
fhat = indices*fhat
ffilt = np.fft.ifft(fhat)

In [ ]:
# small sanity check: our new spectrum looks like this: nice, huh?
plt.plot(PSDclean)
plt.ylim(1e-2,1e5)
plt.axhline(y=1e0, color="r")
plt.semilogy()

In [ ]:
solar_ts_series_corrected_variance.plot()
plt.ylim(-4,4)
pd.Series(ffilt, solar_ts_series_corrected_variance.index).plot()
plt.ylim(-4,4)
plt.legend(["original", "after fft"])

# GARCH1


In [ ]:
# solar_ts_series is demeaned but not corrected for increasing variance (which is why we deploy the GARCH model in the first place)

In [ ]:
import pyflux as pf


In [ ]:
returns = pd.DataFrame(np.diff(solar_ts_series))


In [ ]:
returns.plot()

In [ ]:
model = pf.GARCH(solar_ts_series,p=1,q=1)
x = model.fit()
x.summary()

In [ ]:
model.plot_fit()

In [ ]:
model.plot_predict(h=10)

# Extreme Value Theory

In [ ]:
from pyextremes import __version__, get_extremes
from pyextremes.plotting import plot_extremes
from pyextremes import EVA
print("pyextremes", __version__)

In [ ]:
# "In order for the analysis results to be meaningful, data needs to be pre-processed by the user. 
# This may include removal of data gaps, detrending, interpolation, removal of outliers, etc."
# ==> !! Data needs to be detrended! TODO: Does this also imply constant variance?
# I assume yes and take solar_ts_series_corrected_variance as in input
# TODO: So what exactly does pyextremes expect from the time series?
from pyextremes import EVA

solar_ts_series_corrected_variance

model = EVA(solar_ts_series_corrected_variance)
model

In [ ]:
model.get_extremes(method="BM", block_size="365.2425D")

In [ ]:
model

In [ ]:
model.fit_model()

In [ ]:
model

In [ ]:
summary = model.get_summary(
    return_period=[1, 2, 5, 10, 25, 50, 100, 250, 500, 1000],
    alpha=0.95,
    n_samples=1000,
)

In [ ]:
print(summary)

In [ ]:
model.plot_diagnostic(alpha=0.95)

In [ ]:
extremes = get_extremes(
    ts=solar_ts_series_corrected_variance,
    method="BM",
    extremes_type="high",
    block_size="365.2425D",
    errors="raise",
    min_last_block=None,
)

plot_extremes(
    ts=solar_ts_series_corrected_variance,
    extremes=extremes,
    extremes_method="BM",
    extremes_type="low",
    block_size="365.2425D",
)
extremes

In [ ]:
model.get_extremes("POT", threshold=3, r="12H")


In [ ]:
model.plot_extremes(show_clusters=True)

# Gaussian Process

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

In [ ]:
train_size = int(len(solar_ts_series_corrected_variance) * 0.7)
ts_train = solar_ts_series_corrected_variance[:train_size]
ts_test = solar_ts_series_corrected_variance[train_size:]

In [ ]:
X_train = ts_train.values.reshape(-1,1)
Y_train = ts_test.values.reshape(-1,1)

In [ ]:
kernel = RBF(length_scale=1)
model = GaussianProcessRegressor(kernel=kernel)
model.fit(X_train, Y_train)

In [ ]:
# end of training data: x_t. Before: window, after: horizon
# "predicting a number" = regression problem

In [ ]:
# evtl. nützlich um Modelle zu vergleichen

def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true, y_pred)

  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy(),
          "mase": mase.numpy()}


# consider using the aggregating version (which loses information though - do we really want that?)
def evaluate_preds(y_true, y_pred):
  # Make sure float32 (for metric calculations)
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate various metrics
  mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  mse = tf.keras.metrics.mean_squared_error(y_true, y_pred)
  rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred)
  mase = mean_absolute_scaled_error(y_true, y_pred)

  # Account for different sized metrics (for longer horizons, reduce to single number)
  if mae.ndim > 0: # if mae isn't already a scalar, reduce it to one by aggregating tensors to mean
    mae = tf.reduce_mean(mae)
    mse = tf.reduce_mean(mse)
    rmse = tf.reduce_mean(rmse)
    mape = tf.reduce_mean(mape)
    mase = tf.reduce_mean(mase)

  return {"mae": mae.numpy(),
          "mse": mse.numpy(),
          "rmse": rmse.numpy(),
          "mape": mape.numpy(),
          "mase": mase.numpy()}
     

In [ ]:
HORIZON = 1 # predict 1 step at a time
WINDOW_SIZE = 7 # use a week worth of timesteps to predict the horizon

# Create function to label windowed data
def get_labelled_windows(x, horizon=1):
  """
  Creates labels for windowed dataset.

  E.g. if horizon=1 (default)
  Input: [1, 2, 3, 4, 5, 6] -> Output: ([1, 2, 3, 4, 5], [6])
  """
  return x[:, :-horizon], x[:, -horizon:]


# Test out the window labelling function
test_window, test_label = get_labelled_windows(tf.expand_dims(tf.range(8)+1, axis=0), horizon=HORIZON)
print(f"Window: {tf.squeeze(test_window).numpy()} -> Label: {tf.squeeze(test_label).numpy()}")

# Create function to view NumPy arrays as windows
def make_windows(x, window_size=7, horizon=1):
  """
  Turns a 1D array into a 2D array of sequential windows of window_size.
  """
  # 1. Create a window of specific window_size (add the horizon on the end for later labelling)
  window_step = np.expand_dims(np.arange(window_size+horizon), axis=0)
  # print(f"Window step:\n {window_step}")

  # 2. Create a 2D array of multiple window steps (minus 1 to account for 0 indexing)
  window_indexes = window_step + np.expand_dims(np.arange(len(x)-(window_size+horizon-1)), axis=0).T # create 2D array of windows of size window_size
  # print(f"Window indexes:\n {window_indexes[:3], window_indexes[-3:], window_indexes.shape}")

  # 3. Index on the target array (time series) with 2D array of multiple window steps
  windowed_array = x[window_indexes]

  # 4. Get the labelled windows
  windows, labels = get_labelled_windows(windowed_array, horizon=horizon)

  return windows, labels


# Consider using tf.keras.preprocessing.timeseries_dataset_from_array() instead!

# Make the train/test splits
def make_train_test_splits(windows, labels, test_split=0.2):
  """
  Splits matching pairs of windows and labels into train and test splits.
  """
  split_size = int(len(windows) * (1-test_split)) # this will default to 80% train/20% test
  train_windows = windows[:split_size]
  train_labels = labels[:split_size]
  test_windows = windows[split_size:]
  test_labels = labels[split_size:]
  return train_windows, test_windows, train_labels, test_labels

import os

# Create a function to implement a ModelCheckpoint callback with a specific filename
def create_model_checkpoint(model_name, save_path="model_experiments"):
  return tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path, model_name), # create filepath to save model
                                            verbose=0, # only output a limited amount of text
                                            save_best_only=True) # save only the best model to file

def make_preds(model, input_data):
  """
  Uses model to make predictions on input_data.

  Parameters
  ----------
  model: trained model
  input_data: windowed input data (same kind of data model was trained on)

  Returns model predictions on input_data.
  """
  forecast = model.predict(input_data)
  return tf.squeeze(forecast) # return 1D array of predictions

In [ ]:
# optional (dense)

import tensorflow as tf
from tensorflow.keras import layers

# Set random seed for as reproducible results as possible
tf.random.set_seed(42)

# Construct model
model_1 = tf.keras.Sequential([
  layers.Dense(128, activation="relu"),
  layers.Dense(HORIZON, activation="linear") # linear activation is the same as having no activation
], name="model_1_dense") # give the model a name so we can save it

# Compile model
model_1.compile(loss="mae",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["mae"]) # we don't necessarily need this when the loss function is already MAE

# Fit model
model_1.fit(x=train_windows, # train windows of 7 timesteps of Bitcoin prices
            y=train_labels, # horizon value of 1 (using the previous 7 timesteps to predict next day)
            epochs=100,
            verbose=1,
            batch_size=128,
            validation_data=(test_windows, test_labels),
            callbacks=[create_model_checkpoint(model_name=model_1.name)]) # create ModelCheckpoint callback to save bes

# maybe consider using EarlyStopping instead? 
# Load in saved best performing model_1 and evaluate on test data
model_1 = tf.keras.models.load_model("model_experiments/model_1_dense")
model_1.evaluate(test_windows, test_labels)

# Make predictions using model_1 on the test dataset and view the results
model_1_preds = make_preds(model_1, test_windows)
len(model_1_preds), model_1_preds[:10]

# Evaluate preds
model_1_results = evaluate_preds(y_true=tf.squeeze(test_labels), # reduce to right shape
                                 y_pred=model_1_preds)
model_1_results

In [ ]:
# model 2: use different WINDOW_SIZE

In [ ]:
# model 3: horizon > 1 ==> N nodes in the last dense layer, unrolled by make_preds (supposedly)

In [ ]:
# NBEATS
# Create NBeatsBlock custom layer
class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self, # the constructor takes all the hyperparameters for the layer
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs): # the **kwargs argument takes care of all of the arguments for the parent class (input_shape, trainable, name)
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    # Block contains stack of 4 fully connected layers each has ReLU activation
    self.hidden = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
    # Output of block is a theta layer with linear activation
    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs): # the call method is what runs when the layer is called
    x = inputs
    for layer in self.hidden: # pass inputs through each hidden layer
      x = layer(x)
    theta = self.theta_layer(x)
    # Output the backcast and forecast from theta
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [ ]:
ts_train.index

In [ ]:
X_train[0]

In [ ]:
#train_size = int(len(solar_ts_series_corrected_variance) * 0.7)
#ts_train = solar_ts_series_corrected_variance[:train_size]
#ts_test = solar_ts_series_corrected_variance[train_size:]

# Add windowed columns
watts_nbeats = solar_ts_series_corrected_variance.to_frame("Leistung")
for i in range(WINDOW_SIZE):
  watts_nbeats[f"Leistung+{i+1}"] = watts_nbeats["Leistung"].shift(periods=i+1)
watts_nbeats.dropna().head()

X = watts_nbeats.dropna().drop("Leistung", axis=1)
y = watts_nbeats.dropna()["Leistung"]

# Make train and test sets
split_size = int(len(X) * 0.8)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]
len(X_train), len(y_train), len(X_test), len(y_test)



#X_train = solar_ts_series_corrected_variance.index.to_numpy()
#X_train = np.array([x.to_pydatetime() for x in X_train]).astype('datetime64[ns]')
#X_test = solar_ts_series_corrected_variance.index.to_numpy()
#X_test = np.array([x.to_pydatetime() for x in X_test]).astype('datetime64[ns]')
#Y_train = ts_train.values.reshape(-1,1)
#Y_test = ts_test.values.reshape(-1,1)



# 1. Turn train and test arrays into tensor Datasets
train_features_dataset = tf.data.Dataset.from_tensor_slices(X_train)
train_labels_dataset = tf.data.Dataset.from_tensor_slices(Y_train)

test_features_dataset = tf.data.Dataset.from_tensor_slices(X_test)
test_labels_dataset = tf.data.Dataset.from_tensor_slices(y_test)

# 2. Combine features & labels
train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

# 3. Batch and prefetch for optimal performance
BATCH_SIZE = 1024 # taken from Appendix D in N-BEATS paper
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

train_dataset, test_dataset

In [ ]:
watts_nbeats

In [ ]:
X_train

In [ ]:
dt_object = datetime.fromtimestamp(X_train[0])

In [ ]:
X_train

# Naive Model and Backtesting

In [ ]:
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor

In [ ]:
# Data Preparation for Naive Model
ts = solar_ts_series_corrected_variance
ts = ts.dropna()

In [ ]:
# Moving average model
def moving_average(data: pd.DataFrame, window_size: int=4*3, shift_size: int=96):
    moving_avg = data.rolling(window=window_size).mean()
    shifted_moving_avg = moving_avg.shift(shift_size)
    return(shifted_moving_avg)

In [ ]:
# Plot Naive Model
naive_model = moving_average(ts)

test_date_start = '2024-01-01 00:00+00:00'
test_ts = ts[test_date_start:]
naive_model_print = naive_model[test_date_start:]

plt.figure(figsize=(12, 6))
plt.plot(test_ts.index, test_ts, label='Original')
plt.plot(naive_model_print.index, naive_model_print, label='Moving average', linestyle='--')
plt.legend()
plt.title('Naive Model v.s. Original Data')
plt.xlabel('Date')
plt.ylabel('Time Series')
plt.show()

In [ ]:
class TimeSeriesPredictionModel():
    """
    Time series prediction model implementation
    
    Parameters
    ----------
        model_name : class
            Choice of regressor
        model_params : dict
            Definition of model specific tuning parameters
    
    Functions
    ----------
        train : Train chosen model
        forcast : Apply trained model to prediction period and generate forecast DataFrame
    """
    def __init__(self, model_name, 
                 model_params: dict) -> None:
        """Initialize a new instance of time_series_prediction_model."""
        self.model = model_name(**model_params) 
    
    def train(self, X_train: pd.DataFrame, y_train: pd.Series) -> None:
        """Train chosen model."""
        self.X_train = X_train
        self.y_train = y_train
        self.model.fit(self.X_train, self.y_train)
    
    def forecast(self, X_test: pd.DataFrame) -> pd.DataFrame:
        """Apply trained model to prediction period and generate forecast DataFrame."""
        self.X_test = X_test
        forecast_df = pd.DataFrame(self.model.predict(self.X_test), index=self.X_test.index)
        forecast_df.index.name = 'Datum'
        return forecast_df

In [ ]:
# Dummy data preparation for TimeSeriesPredictionModel

data = pd.DataFrame(index=ts.index, columns=['25h_lag', '24h_lag', 'Original'])
data['Original'] = ts
data['24h_lag'] = ts.shift(96)
data['25h_lag'] = ts.shift(100)

data

In [ ]:
test_date_start = '2024-01-01 00:00+00:00'

In [ ]:
# Dummy Data train-test split
train_df = data[:test_date_start]
train_df = train_df.drop(train_df.tail(1).index)
X_train = train_df[['25h_lag', '24h_lag']]
y_train = train_df[['Original']]

test_df = data[test_date_start:]
X_test = test_df[['25h_lag', '24h_lag']]
y_test = test_df[['Original']]

In [ ]:
# Backtesting mit Sliding Window

def backtesting(X_train: pd.DataFrame, y_train: pd.DataFrame,
                X_test: pd.DataFrame, y_test: pd.DataFrame,
                model: TimeSeriesPredictionModel, prediction_step_size: int=96):

    # initializing output df
    predictions = pd.DataFrame(index=y_test.index, columns=['Original', 'Predictions'])
    predictions['Original'] = y_test

    for i in range(0, len(X_test)-prediction_step_size, prediction_step_size):
        end_idx = i + prediction_step_size
        forecast_index= X_test.iloc[i:end_idx].index
        
        # fit model and predict
        model.train(X_train, y_train)
        forecast = model.forecast(X_test.iloc[i:end_idx])
        predictions.loc[forecast_index, 'Predictions'] = forecast.to_numpy()
    
        print(f'Finished Forecast for {forecast_index[-1].date()}')

        # delete old time window from train data
        X_train = X_train.drop(X_train.head(prediction_step_size).index)
        y_train = y_train.drop(y_train.head(prediction_step_size).index)

        # add next time window to train data
        X_train = pd.concat([X_train, X_test.iloc[i:end_idx]])
        y_train = pd.concat([y_train, y_test.iloc[i:end_idx]])

    return predictions

In [ ]:
# Initializing random forest regressor as instance of TimeSeriesPredictionModel
rdnf = TimeSeriesPredictionModel(RandomForestRegressor, {'n_estimators': 10, 'criterion': 'squared_error', 'max_depth': 5})

In [ ]:
rdnf_pred = backtesting(X_train, y_train, X_test, y_test, rdnf)

In [ ]:
rdnf_pred = rdnf_pred.dropna() # ausgehend vom ersten Testzeitraum werden nur vollständige Test-Perioden predicted
rdnf_pred.plot()

In [ ]:
# Metrics

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

def evaluation(y_true, y_pred):
    
    mae = mean_absolute_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)

    return mae, mape, mse, r2

In [ ]:
mae, mape, mse, r2 = evaluation(rdnf_pred['Original'], rdnf_pred['Predictions'])

print(f'Model: Random Forest \n Mean absolute error: {mae}\n Mean absolute percentage error: {mape} \n Mean squared error: {mse} \n r2_score: {r2}')

In [ ]:
mae, mape, mse, r2 = evaluation(test_ts, naive_model_print)

print(f'Model: Naive Moving Average \n Mean absolute error: {mae}\n Mean absolute percentage error: {mape} \n Mean squared error: {mse} \n r2_score: {r2}')